Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 3.4 ms, sys: 318 µs, total: 3.72 ms
Wall time: 10.3 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/7309/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 233 ms, sys: 24.2 ms, total: 257 ms
Wall time: 2.45 s


[1.3035453437342577,
 1.8062460169463281,
 1.066878539792861,
 1.7715517109133814,
 1.7988407611671402,
 1.1932156129745255,
 1.4010196002936897,
 1.9098389902118176,
 1.4898975446627034,
 1.7110379719713087,
 1.0027014657320819,
 1.8849990348152037,
 1.8289437150605727,
 1.1744384514943438,
 1.4007443619675308,
 1.9484042917226887,
 1.7017986832503769,
 1.233111282158142,
 1.1374489005634496,
 1.7486853175848456,
 1.9924201096902339,
 1.020085807094227,
 1.7218603450004855,
 1.144974011995472,
 1.2959721087902059,
 1.6927218951316116,
 1.8105787181168829,
 1.4313351658321716,
 1.7019999794707643,
 1.9780574555348984,
 1.8044082552089589,
 1.2387222374121596,
 1.4452957080936466,
 1.5471254231917535,
 1.412707059359279,
 1.0036362692448972,
 1.88124963866056,
 1.541302218972449,
 1.7718418246835008,
 1.3032631300111333,
 1.2388607615507636,
 1.277501670649133,
 1.6510417152296275,
 1.4247501721179674,
 1.3259485358927043,
 1.12525725140017,
 1.4272489111610898,
 1.9606406185919658,
 1.

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.4915711502281979,
 1.1428610682329428,
 1.567818373504867,
 1.3512018394490957,
 1.0716956265013309,
 1.7172181257167873,
 1.48970332489997,
 1.4230018668895426,
 1.94358744680879,
 1.7124582464693194,
 1.1325959645761356,
 1.23413196172275,
 1.9525709887657456,
 1.1109113660782617,
 1.4945044136235746,
 1.1205090210660118,
 1.051500824263921,
 1.8052520830116823,
 1.0174315265613343,
 1.8871326730053877,
 1.945813033983205,
 1.3516603249058807,
 1.9429719451649774,
 1.4448861986493289,
 1.1182852202925366,
 1.398191509043675,
 1.5869643039894388,
 1.3999119584899118,
 1.9370408516395443,
 1.8588699527182557,
 1.7849929722931783,
 1.0644525132308096,
 1.7244765999555647,
 1.9555812698560189,
 1.8141370348481047,
 1.1754472555661217,
 1.9973873676925562,
 1.433113401011787,
 1.8705657741352573,
 1.7007960288672304,
 1.3967981549750126,
 1.735998595869344,
 1.287499611289017,
 1.142597094300092,
 1.5189947703483613,
 1.047028068346882]

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 159 ms, sys: 24.5 ms, total: 183 ms
Wall time: 1.81 s
